In [16]:
!pip install -U langchain-openai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [5]:
import os
from google.colab import userdata

# Load secrets from Colab's secret manager
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get('AZURE_OPENAI_ENDPOINT')
os.environ["OPENAI_API_VERSION"] = userdata.get('OPENAI_API_VERSION')
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = userdata.get('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')

## Simple Prompting

In [8]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"]
)


template = """Sentence: {sentence}
Translation in {language}: """
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])
formatted_prompts = prompt.format(sentence="Google's service, offered free of charge, instantly translates words, phrases, and web pages between English and over 100 other languages.", language="Russian")
response = llm.invoke(formatted_prompts)
response.content

'Сервис Google, предлагаемый бесплатно, мгновенно переводит слова, фразы и веб-страницы с английского на более чем 100 других языков.'

## Load Data

In [13]:
import pandas as pd

csv_data = """Name,Age,City
John,25,New York
Emily,28,Los Angeles
Michael,22,Chicago
"""

with open('sample.csv', 'w') as f:
  f.write(csv_data)

sample = pd.read_csv('sample.csv')

In [14]:
display(sample)

,Name,Age,City
0,John,25,New York
1,Emily,28,Los Angeles
2,Michael,22,Chicago


In [17]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(file_path='sample.csv')
data = loader.load()
print(data)

[Document(metadata={'source': 'sample.csv', 'row': 0}, page_content='Name: John\nAge: 25\nCity: New York'), Document(metadata={'source': 'sample.csv', 'row': 1}, page_content='Name: Emily\nAge: 28\nCity: Los Angeles'), Document(metadata={'source': 'sample.csv', 'row': 2}, page_content='Name: Michael\nAge: 22\nCity: Chicago')]


## Data Connections w/ Semantic Splitting

In [19]:
import pandas as pd

txt_data = """Amid quiet land, tall mountains rise like strong guards of nature's peace.
Cool, fresh air brings calm, while leaves move and make soft forest sounds.
The hills wear green and brown, like art from nature, lovely to the eyes.
When the sun comes up, it shines gold on the peaks, lighting a wild world.
"""

with open('mountain.txt', 'w') as f:
  f.write(txt_data)

print(pd.read_csv('mountain.txt'))

                                             Amid quiet land  \
Cool                                   fresh air brings calm   
The hills wear green and brown          like art from nature   
When the sun comes up            it shines gold on the peaks   

                                tall mountains rise like strong guards of nature's peace.  
Cool                               while leaves move and make soft forest sounds.          
The hills wear green and brown                                lovely to the eyes.          
When the sun comes up                                      lighting a wild world.          


In [21]:
with open("mountain.txt") as f:
    mountain = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = len
)

texts = text_splitter.create_documents([mountain])

print(texts[0])
print(texts[1])

page_content='Amid quiet land, tall mountains rise like strong guards of nature's peace.'
page_content='Cool, fresh air brings calm, while leaves move and make soft forest sounds.'
